In [75]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import pandas as pd

mask_directory = '/Users/kathum/Research/COPEX/sink_region_files/masks'
WAM_files = '/Volumes/LaCie_10TB_Keys_KH/era5/output/backtrack/co_altclimate_nfr_2000_2023/'
sink_region_name = 'nfr'

In [76]:
def make_mask(region):
    region_csvpath = '/Users/kathum/Research/COPEX/sink_region_files/source_region_information.csv'
    region_info = pd.read_csv(region_csvpath)
    region_filepath = region_info.loc[region_info['region_name']== region, 'mask_filepath'].values
    
    #read in the mask associated with this region
    if region.capitalize() == 'Colorado':
        mask = xr.open_dataarray(region_filepath[0])
        #remove the sink region from this mask
        remove_mask = xr.open_dataarray(region_info.loc[region_info['other_acronym']== sink_region_name, 'mask_filepath'].item())
        mask = mask - remove_mask
        mask = mask.where(mask>0, other = 0)
        remove_mask.close()
    elif region.lower() == 'western us':
        #remove colorado and the sink region from this mask
        mask = xr.open_dataarray(region_filepath[0])
        for remove_region in [region_info.loc[region_info['other_acronym']== sink_region_name, 'region_name'].item(), 'Colorado']:
            remove_mask = xr.open_dataarray(region_info.loc[region_info['region_name']== remove_region, 'mask_filepath'].item())
            mask = mask - remove_mask
        mask = mask.where(mask == 1, other = 0)
        remove_mask.close()
    elif region== 'Other Water':
        mask = 1 - xr.open_dataarray('/Users/kathum/Research/COPEX/sink_region_files/masks/land_source_region.nc') 
        mask = mask.squeeze()
        water_mask_paths = region_info.loc[(region_info['surface_type'].notnull()), ['mask_filepath']].values.squeeze()
        for path in water_mask_paths:
            remove_mask =  xr.open_dataarray(path)
            mask = mask - remove_mask
        remove_mask.close()
        mask = mask.where(mask>0, other = 0)
    elif region == 'Other Land':
        mask = xr.open_dataarray('/Users/kathum/Research/COPEX/sink_region_files/masks/land_source_region.nc')
        mask = mask.squeeze()
        land_mask_paths = region_info.loc[(region_info['surface_type'].notnull()), ['mask_filepath']].values.squeeze()
        for path in land_mask_paths:
            remove_mask =  xr.open_dataarray(path)
            mask = mask - remove_mask
        remove_mask.close()
        mask = mask.where(mask>0, other = 0)    
    else:
        mask = xr.open_dataarray(region_filepath[0])
    
    return mask

## Create CSV

In [78]:
region_csvpath = '/Users/kathum/Research/COPEX/sink_region_files/source_region_information.csv'
region_info = pd.read_csv(region_csvpath)
files = sorted(glob.glob(WAM_files+'*.nc'))[:-80][:11] #NC files to be used and removing wam output from after Sept 30th
regions = ['Other Water','Pacific Ocean','Gulf of California', 'Atlantic Ocean','Gulf of Mexico', 'Other Land', 'Western US', 'Southern US','Midwestern US','Colorado',
           'Eastern Plains','Front Range','Pikes Peak','Arkansas Basin','High Mountain Valley']

data_dict = {}
for i, file in enumerate(files):
    day_data = xr.open_dataset(file)['e_track']
    data_dict[i] = {'date':pd.date_range(file[-19:-9], periods=1)[0]}
    for region in regions:
        region_mask = make_mask(region)
        day_region_e_track = (day_data * region_mask).sum(dim=['latitude','longitude']).values
        data_dict[i][region+'_t0_e_track'] = day_region_e_track
    day_data.close()
        
data_df = pd.DataFrame(data_dict).transpose()
data_df

,date,Other Water_t0_e_track,Pacific Ocean_t0_e_track,Gulf of California_t0_e_track,Atlantic Ocean_t0_e_track,Gulf of Mexico_t0_e_track,Other Land_t0_e_track,Western US_t0_e_track,Southern US_t0_e_track,Midwestern US_t0_e_track,Colorado_t0_e_track,Eastern Plains_t0_e_track,Front Range_t0_e_track,Pikes Peak_t0_e_track,Arkansas Basin_t0_e_track,High Mountain Valley_t0_e_track
0,2000-01-01 00:00:00,8042.574794012333,4349084.9512779005,6382.884692013264,854599.611052634,1934.162124201026,58054.98983023991,1980851.4313719848,170.94983419842902,3696.024621677585,441950.99205103144,28318.34949232638,170407.81185913086,9301.073272705078,431.29907923564315,64577.11827087402
1,2000-01-02 00:00:00,10999.343512942602,4396599.590817767,21491.728424072266,1084663.6694921993,2495.4507534109957,85962.74998815231,1259058.010005609,486.02797481755584,1022.3137007427438,1409131.1007039547,57711.698968052864,547892.2291259766,106042.31065368652,24271.44216632843,290040.53857421875
2,2000-01-03 00:00:00,14548.7446995612,4230650.447054281,53290.88123321533,1214330.4178286134,18742.466079319365,122390.02327813875,331804.1503557553,4643.959433681106,5226.424684257581,121880.05709613115,69238.96623402834,374064.3601074219,6068.854589223862,560.3822497427464,9059.644638359547
3,2000-01-04 00:00:00,21122.603855161185,4775540.300822457,88394.01195144653,1426604.9541975695,272076.8367540854,103437.9344366421,67080.77676261656,6334.133637427953,607.2230908203346,1783.45728765917,3.2697471282444894,129.98116207122803,0.19600526406429708,1.500222128815949,11.096128918230534
4,2000-01-05 00:00:00,25003.287201364623,4133948.988827388,45780.902789115906,1947277.1007539988,164996.2385136038,159587.57599530154,58110.68821372977,3089.3809700452257,90.0306283678583,26438.46329063922,112.38611923158169,6105.731971740723,45.46668644249439,7.35404259711504,1679.2839056253433
5,2000-01-06 00:00:00,27576.369159769518,2429489.75518734,67756.3152809143,2216625.3949880344,22745.555118783377,215719.61534418614,51947.74305288798,2200.291693932377,238.0850709428778,1597.4688034455758,12.02241649699863,236.73078398406506,14.13300265185535,2.7843430715147406,194.26588416099548
6,2000-01-07 00:00:00,42593.07815727464,3935498.1145563065,142535.16226959229,2387792.754305978,14701.92372217805,275828.0250824531,85534.85398034929,1091.1872005156474,67.12462104366568,2788.837926107575,1.0572599553561304,18.931177588179708,2.6518161662388593,0.06457263595075347,76.87940175458789
7,2000-01-08 00:00:00,57441.77782880506,7984396.288908076,150398.41175079346,2361945.63964983,13134.126175660494,352484.67217107734,443676.1755504165,109.27793685007782,5.63764009251463,14836.21809872362,0.40026315342402086,752.2706138342619,1.109826194238849,0.07999171537812799,63.4867495931685
8,2000-01-09 00:00:00,77294.6319914376,4029994.6945116073,188595.7807006836,2513580.082935631,36094.240629256274,588120.9333217324,960103.6107467344,3321.914777694596,78.14784279446425,47532.93803769024,1250.4667376205325,14586.220891401172,0.6868686880916357,3.5522514856420457,908.7929541710764
9,2000-01-10 00:00:00,86759.63644366746,4354208.993551242,209858.82024383545,2278232.8913551513,140707.42904943955,722402.7351684009,660646.6668861709,6641.355775859491,38.97377129130633,236631.65448587388,2092.039039088413,135100.8170928955,2.4616838339716196,8.773302745074034,7181.662964813411
